this is another messy notebook! this one is used to study jet trscer volume fraction

In [ ]:
import glob
import os
from natsort import natsorted
from postprocessing import *

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
simulation_directory_jet = str('/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d100_m12/jet41_early')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

In [ ]:
i_file = 5
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory_jet + filename, "r")

In [ ]:
x = snap_data['PartType0/Jet_Tracer'][:]

In [ ]:
k1 = snap_data['Config'].attrs['BH_CONSTANT_POWER_RADIOMODE_IN_ERG_PER_S']
print(k1)

In [ ]:
plt.hist(np.log10(x[x>1e-25]), bins=100, log=True)
plt.axvline(-3, c='black', zorder=100)
plt.xlabel('jet tracer')

In [ ]:
density_arr  = np.array([30, 60, 100, 200, 300])
jetpower_arr = np.array([41, 42, 43, 44])
machnum_arr  = np.array([4, 8, 12])

In [ ]:
threshold_tracer = 1e-3

In [ ]:
def calculate_jet_tracer_dj(i_file, density_arr=density_arr, jetpower_arr=jetpower_arr, mach=8):
    """
    Create a table with jet tracer volume fraction for different density and jet power values
    Input: i_file       (snapshot number)
           density_arr  (array of number densities for which simulations exist)
           jetpower_arr (array of jet power values for which simulations exist)
           mach         (mach number)
    Output: array with three columns: number density, jet power, and jet volume fraction. and time of the snapshot in Myr
    
    """
    
    result = []
    for density in density_arr:
        for jetpower in jetpower_arr:
            
            simulation_directory_jet = str(f'/n/holyscratch01/hernquist_lab/borodina/turb_drive_center_d{density}_m{mach}/jet{jetpower}_early')
            output_directory_jet = simulation_directory_jet + "/output/"
            filename = "snap_%03d.hdf5" % (i_file)
            try:
                snap_data = h5py.File(output_directory_jet + filename, "r")
                volume = snap_data['PartType0/Masses'][:]/snap_data['PartType0/Density'][:] 
                tracer = snap_data['PartType0/Jet_Tracer'][:]
                jet_volume = volume[tracer > threshold_tracer]
                jet_volume_fraction = np.sum(jet_volume)/np.sum(volume)
                result.append([density, jetpower, jet_volume_fraction])
            except:
                result.append([density, jetpower, -1])
    time = get_time_from_snap(snap_data)
    return np.array(result).T, time * unit_time_in_megay

In [ ]:
xaxis = np.array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5])

for i in [0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
    result, time = calculate_jet_tracer_dj(i)
    
    fig, ax = plt.subplots(figsize=(4.5, 3))
    norm=plt.Normalize(np.log10(5e-4), np.log10(0.2))

    mask = (result[2] > 0)

    sc = ax.scatter(xaxis[mask], result[1][mask], c=np.log10(result[2][mask]), 
                    cmap='plasma', norm=norm, marker='s', s=1580, zorder=100)
    ax.tick_params(labelsize=12, direction='in')

    locs, labels = plt.xticks()  # Get the current locations and labels.
    plt.xticks([1, 2, 3, 4, 5], density_arr)  # Set text labels and properties.
    ax.set_xlim(0.5, 5.5)
    ax.set_ylim(40.5, 44.5)

    ax.set_xlabel('density [cm$^{-3}$]', fontsize=15)
    ax.set_ylabel('log (jet power)', fontsize=15)

    cbar = plt.colorbar(sc, format='%g')
    cbar.set_label(label=r'log(jet tracer volume fraction)', size=10)

    ax.set_title("t=%.2f Myr"%(time * unit_time_in_megayr))
    plt.savefig( f'./figures/jet_tracer_dj_{i}.png', bbox_inches='tight', dpi=300)
    plt.close()

In [ ]:
def calculate_jet_tracer_mnum(i_file, machnum_arr=machnum_arr, jetpower_arr=jetpower_arr, density=100):
    """
    Create a table with jet tracer volume fraction for different mach numbers and jet power values
    Input: i_file       (snapshot number)
           machnum_arr  (array of mach numbers for which simulations exist)
           jetpower_arr (array of jet power values for which simulations exist)
           density      (number density of the simulation)
           
    Output: array with three columns: mach number, jet power, and jet volume fraction. and time of the snapshot in Myr
    
    """
    result = []
    for machnum in machnum_arr:
        for jetpower in jetpower_arr:
            simulation_directory_jet = str(f'/n/holyscratch01/hernquist_lab/borodina/turb_drive_center_d{density}_m{machnum}/jet{jetpower}_early')
            output_directory_jet = simulation_directory_jet + "/output/"
            filename = "snap_%03d.hdf5" % (i_file)
            try:
                snap_data = h5py.File(output_directory_jet + filename, "r")
                volume = snap_data['PartType0/Masses'][:]/snap_data['PartType0/Density'][:] 
                tracer = snap_data['PartType0/Jet_Tracer'][:]
                jet_volume = volume[tracer > threshold_tracer]
                jet_volume_fraction = np.sum(jet_volume)/np.sum(volume)
                result.append([machnum, jetpower, jet_volume_fraction])
            except:
                result.append([machnum, jetpower, -1])
    time = get_time_from_snap(snap_data)
    return np.array(result).T, time * unit_time_in_megay

In [ ]:
xaxis = np.array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])
for i in [0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
    result, time = calculate_jet_tracer_mnum(i)

    fig, ax = plt.subplots(figsize=(2.7, 3))
    norm=plt.Normalize(np.log10(5e-4), np.log10(0.2))

    mask = (result[2] > 0)

    sc = ax.scatter(xaxis[mask], result[1][mask], c=np.log10(result[2][mask]), 
                    cmap='plasma', norm=norm, marker='s', s=1580, zorder=100)
    ax.tick_params(labelsize=12, direction='in')

    locs, labels = plt.xticks()  # Get the current locations and labels.
    plt.xticks([1, 2, 3], machnum_arr)  # Set text labels and properties.
    ax.set_xlim(0.5, 3.5)
    ax.set_ylim(40.5, 44.5)

    ax.set_xlabel('Mach number', fontsize=15)
    ax.set_ylabel('log (jet power)', fontsize=15)

    cbar = plt.colorbar(sc, format='%g')
    cbar.set_label(label=r'log(jet tracer volume fraction)', size=10)

    ax.set_title("t=%.2f Myr"%(time * unit_time_in_megayr))
    plt.savefig( f'./figures/jet_tracer_mnum_{i}.png', bbox_inches='tight', dpi=300)
    plt.close()

## make gif

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = './figures/jet_tracer_dj*.png'
ofilename = './figures/jet_tracer-dj.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=140, loop=0)

## time evolution with density

In [ ]:
def jet_and_density(density, machnumber, jetpower, start='early'):
    """
    Create a table with jet tracer volume fraction and central density evolution with time
    Input: density    (number density of the simulation)
           machnumber (mach number of the simulation)
           jetpower   (jet power of the simulation)
           start      (initial snapshot from the turbulent box)
           
    Output: array with three columns: time un Myr, jet tracer volume fraction, and mean number density within central 50pc
    
    """    
    simulation_directory_jet = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d{density}_m{machnumber}/jet{jetpower}_{start}')
    output_directory_jet = simulation_directory_jet + "/output/"
    figures_directory_jet = simulation_directory_jet + "/output/figures/"
    
    threshold_tracer = 1e-3
    result = []
    i_file = -1
    while True:
        i_file += 1
        output_directory_jet = simulation_directory_jet + "/output/"
        filename = "snap_%03d.hdf5" % (i_file)
        try:
            snap_data = h5py.File(output_directory_jet + filename, "r")
        except:
            break
        time = get_time_from_snap(snap_data) * unit_time_in_megayr

        volume = snap_data['PartType0/Masses'][:]/snap_data['PartType0/Density'][:]
        tracer = snap_data['PartType0/Jet_Tracer'][:]
        jet_volume = volume[tracer > threshold_tracer]
        jet_volume_fraction = np.sum(jet_volume)/np.sum(volume)

        x, y, z = snap_data['PartType0/Coordinates'][:].T
        mask = (np.sqrt((x - 500) ** 2 + (y - 500) ** 2 + (z - 500) ** 2) < 50)
        n_mean = np.mean(snap_data['PartType0/Density'][mask]) * rho_to_numdensity
        result.append([time, jet_volume_fraction, n_mean])
    return result

In [ ]:
result_4_early  = np.array(jet_and_density(100,  4, 41, 'early')).T
result_8_early  = np.array(jet_and_density(100,  8, 41, 'early')).T
result_12_early = np.array(jet_and_density(100, 12, 41, 'early')).T

result_4_mid  = np.array(jet_and_density(100,  4, 41, 'mid')).T
result_8_mid  = np.array(jet_and_density(100,  8, 41, 'mid')).T
result_12_mid = np.array(jet_and_density(100, 12, 41, 'mid')).T

result_4_later  = np.array(jet_and_density(100,  4, 41, 'later')).T
result_8_later  = np.array(jet_and_density(100,  8, 41, 'later')).T
result_12_later = np.array(jet_and_density(100, 12, 41, 'later')).T

result_uni = np.array(jet_and_density(100, 8, 42, 'uniform')).T


In [ ]:
len(result_4_later[1])

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,3))
# ax2 = ax1.twinx()



ax1.plot(result_12_early[0], result_12_early[1], label='m=12 jet=42', c='darkblue', ls='--', )
ax1.plot(result_8_early[0], result_8_early[1], label='m=8 j=42', c='darkgreen', ls='--', )
ax1.plot(result_4_early[0], result_4_early[1], label='m=4 j=42', c='darkred', ls='--', )

ax1.plot(result_uni[0], result_uni[1], label='m=0 j=42', c='black', ls='-')

ax1.axhline(0.05, c='gray', ls='--')

ax1.set_xlabel('t [Myr]', fontsize=10)
ax1.set_xlim(0, 3.5)
ax1.set_ylim(1e-4, 0.2)
# ax2.set_ylim(10, 6e3)

ax1.set_ylabel('jet tracer volume fraction (--)', fontsize=10)
# ax2.set_ylabel('mean density inside central 50 pc [cm$^{-3}$](-)', fontsize=10)
ax1.set_yscale('log')
# ax2.set_yscale('log')
plt.legend(fontsize=9, loc='lower right')
plt.show()

In [ ]:
result_early  = np.array(jet_and_density(100,  12, 41, 'early')).T
result_mid  = np.array(jet_and_density(100,  12, 41, 'mid')).T
result_later = np.array(jet_and_density(100,  12, 41, 'later')).T
result_uni = np.array(jet_and_density(100, 8, 41, 'uniform')).T

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,3))


ax1.plot(result_early[0], result_early[1], label='m=12 jet=41', c='darkred', ls='--', )
# ax2.plot(result_100_12_41[0], result_100_12_41[2], label='m=12 jet=41', c='darkblue')

ax1.plot(result_mid[0], result_mid[1], c='darkred', ls='--', )
# ax2.plot(result_100_8_41[0], result_100_8_41[2], label='m=8 j=41', c='darkgreen')

ax1.plot(result_later[0], result_later[1], c='darkred', ls='--', )
# ax2.plot(result_100_4_41[0], result_100_4_41[2], label='m=4 j=41', c='darkred')

ax1.plot(result_uni[0], result_uni[1], label='no turbulence', c='black', ls='-')

ax1.axhline(0.05, c='gray', ls='--')

ax1.set_xlabel('t [Myr]', fontsize=10)
ax1.set_xlim(0, 3.5)
ax1.set_ylim(1e-4, 0.2)
# ax2.set_ylim(10, 6e3)

ax1.set_ylabel('jet tracer volume fraction (--)', fontsize=10)
# ax2.set_ylabel('mean density inside central 50 pc [cm$^{-3}$](-)', fontsize=10)
ax1.set_yscale('log')
# ax2.set_yscale('log')
plt.legend(fontsize=9, loc='lower right')
plt.show()